In [13]:
%cd /scratch/imb/uqjxie6/benchmmarking/DeepHis2Exp/scripts

/scratch/imb/uqjxie6/benchmmarking/DeepHis2Exp/scripts


In [14]:
import sys
from pathlib import Path
import os
import numpy as np
import pandas as pd
from pathlib import Path, PurePath
from typing import Union, Dict, Optional, Tuple, BinaryIO
import h5py
import json
from matplotlib.image import imread
import matplotlib.pyplot as plt

import anndata
from anndata import (
    AnnData,
    read_csv,
    read_text,
    read_excel,
    read_mtx,
    read_loom,
    read_hdf,
)
from anndata import read as read_h5ad
from anndata import read_h5ad
import scanpy as sc
from scanpy import read_visium, read_10x_mtx

import glob
import torch
import torchvision
import pandas as pd 
import scprep as scp
import anndata as ad
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None


def read_visium_alex(
    path: Union[str, Path],
    genome: Optional[str] = None,
    *,
    count_dir: str = "raw_feature_bc_matrix",
    library_id: str = None,
    load_images: Optional[bool] = True,
    source_image_path: Optional[Union[str, Path]] = None,
) -> AnnData:
    path = Path(path)
    adata = read_10x_mtx(path / count_dir)

    adata.uns["spatial"] = dict()

    from h5py import File

    adata.uns["spatial"][library_id] = dict()

    if load_images:
        files = dict(
            tissue_positions_file=path / 'spatial/tissue_positions_list.csv',
            scalefactors_json_file=path / 'spatial/scalefactors_json.json',
            hires_image=path / 'spatial/tissue_hires_image.png',
            lowres_image=path / 'spatial/tissue_lowres_image.png',
        )

        # check if files exists, continue if images are missing
        for f in files.values():
            if not f.exists():
                if any(x in str(f) for x in ["hires_image", "lowres_image"]):
                    logg.warning(
                        f"You seem to be missing an image file.\n"
                        f"Could not find '{f}'."
                    )
                else:
                    raise OSError(f"Could not find '{f}'")

        adata.uns["spatial"][library_id]['images'] = dict()
        for res in ['hires', 'lowres']:
            try:
                adata.uns["spatial"][library_id]['images'][res] = imread(
                    str(files[f'{res}_image'])
                )
            except Exception:
                raise OSError(f"Could not find '{res}_image'")

        # read json scalefactors
        adata.uns["spatial"][library_id]['scalefactors'] = json.loads(
            files['scalefactors_json_file'].read_bytes()
        )

        adata.uns["spatial"][library_id]["metadata"] = {}

        # read coordinates
        positions = pd.read_csv(files['tissue_positions_file'], header=None)
        positions.columns = [
            'barcode',
            'in_tissue',
            'array_row',
            'array_col',
            'pxl_col_in_fullres',
            'pxl_row_in_fullres',
        ]
        positions.index = positions['barcode']

        adata.obs = adata.obs.join(positions, how="left")

        adata.obsm['spatial'] = adata.obs[
            ['pxl_row_in_fullres', 'pxl_col_in_fullres']
        ].to_numpy()
        adata.obs.drop(
            columns=['barcode', 'pxl_row_in_fullres', 'pxl_col_in_fullres'],
            inplace=True,
        )

        # put image path in uns
        if source_image_path is not None:
            # get an absolute path
            source_image_path = str(Path(source_image_path).resolve())
            adata.uns["spatial"][library_id]["metadata"]["source_image_path"] = str(
                source_image_path
            )

    return adata

In [15]:
data_dir1 = "../data/Alex_NatGen/"
data_dir2 = "../data/Breast_Cancer_10x/"

samps1 = ["1142243F", "CID4290", "CID4465", "CID44971", "CID4535"]
samps2 = ["block1", "block2", "FFPE", "CytAssist_FFPE"]

samples1 = {i:data_dir1 + i for i in samps1}
samples2 = {i:data_dir2 + i for i in samps2}

You may need to rename the files in the two directories to match the following format.

In [16]:
%ls {data_dir1 + samps1[0]} #  For Alex_NatGen

1142243F.tif            metadata.csv            spatial/
filtered_count_matrix/  raw_feature_bc_matrix/


In [17]:
%ls {data_dir2 + samps2[0]} # For Breast_Cancer_10x

filtered_feature_bc_matrix/  filtered_feature_bc_matrix.h5  image.tif  spatial/


In [18]:
%%time
adata_dict1 = {name: read_visium_alex(path, library_id = name, source_image_path = path + f"/{name}.tif") for name,path in samples1.items()}
# adata_dict2 = {name: read_visium(path, library_id = name, source_image_path = path + "/image.tif") for name,path in samples2.items()}

adata_dict = {**adata_dict1}

CPU times: user 1min 26s, sys: 1.36 s, total: 1min 27s
Wall time: 1min 28s


In [23]:
adatas = [v for k,v in adata_dict.items()]
adatas
anndata.concat(adatas)

/scratch/imb/uqjxie6/software/stimage5/lib/python3.8/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 24960 × 36601
    obs: 'in_tissue', 'array_row', 'array_col'
    obsm: 'spatial'

In [11]:
import sys
sys.path.append("../models/Hist2ST/")
from utils import read_tiff, get_data
from graph_construction import calcADJ
from collections import defaultdict as dfd

def read_tiff(path):
    Image.MAX_IMAGE_PIXELS = None
    im = Image.open(path)
    imarray = np.array(im)
    # I = plt.imread(path)
    return im

class ViT_Anndata(torch.utils.data.Dataset):
    """Some Information about ViT_SKIN"""
    def __init__(self, adata_dict, train_set, gene_list, train=True,r=4,norm=False,flatten=True,ori=True,adj=True,prune='NA',neighs=4):
        super(ViT_Anndata, self).__init__()

        self.r = 224//r

        names = list(adata_dict.keys())

        self.ori = ori
        self.adj = adj
        self.norm = norm
        self.train = train
        self.flatten = flatten
        self.gene_list = gene_list
        samples = names
        tr_names = train_set
        te_names = list(set(samples)-set(tr_names))

        if train:
            self.names = tr_names
        else:
            self.names = te_names

        print("Eval set: ", te_names)
        
        self.adata_dict = {k: v for k, v in adata_dict.items() if k in self.names}
    
        print('Loading imgs...')
        self.img_dict = {i:torch.Tensor(np.array(self.get_img(i))) for i in self.names}

        self.gene_set = list(gene_list)
        if self.norm:
            self.exp_dict = {
                i:sc.pp.scale(scp.transform.log(scp.normalize.library_size_normalize(m.to_df()[self.gene_set].values))).astype(np.float64)
                for i,m in self.adata_dict.items()
            }
        else:
            self.exp_dict = {
                i:scp.transform.log(scp.normalize.library_size_normalize(m.to_df()[self.gene_set].values)).astype(np.float64) 
                for i,m in self.adata_dict.items()
            }
        if self.ori:
            self.ori_dict = {i:m.to_df()[self.gene_set].values.astype(np.float64) for i,m in self.adata_dict.items()}
            self.counts_dict={}
            for i,m in self.ori_dict.items():
                n_counts=m.sum(1)
                sf = n_counts / np.median(n_counts)
                self.counts_dict[i]=sf.astype(np.float64)
        self.center_dict = {
            i:np.floor(m.obsm["spatial"]).astype(int)
            for i,m in self.adata_dict.items()
        }
        self.loc_dict = {i:m.obs[['array_col', 'array_row']].values for i,m in self.adata_dict.items()}
        self.adj_dict = {
            i:calcADJ(m,neighs,pruneTag=prune)
            for i,m in self.loc_dict.items()
        }
        self.patch_dict=dfd(lambda :None)
        self.lengths = [i.n_obs for i in self.adata_dict.values()]
        self.cumlen = np.cumsum(self.lengths)
        self.id2name = dict(enumerate(self.names))


    def filter_helper(self):
        a = np.zeros(len(self.gene_list))
        n = 0
        for i,exp in self.exp_dict.items():
            n += exp.shape[0]
            exp[exp>0] = 1
            for j in range((len(self.gene_list))):
                a[j] += np.sum(exp[:,j])


    def __getitem__(self, index):
        ID=self.id2name[index]
        im = self.img_dict[ID].permute(1,0,2)

        exps = self.exp_dict[ID]
        if self.ori:
            oris = self.ori_dict[ID]
            sfs = self.counts_dict[ID]
        adj=self.adj_dict[ID]
        centers = self.center_dict[ID]
        loc = self.loc_dict[ID]
        patches = self.patch_dict[ID]
        positions = torch.LongTensor(loc)
        patch_dim = 3 * self.r * self.r * 4
        exps = torch.Tensor(exps)
        if patches is None:
            n_patches = len(centers)
            if self.flatten:
                patches = torch.zeros((n_patches,patch_dim))
            else:
                patches = torch.zeros((n_patches,3,2*self.r,2*self.r))

            for i in range(n_patches):
                center = centers[i]
                x, y = center
                patch = im[(x-self.r):(x+self.r),(y-self.r):(y+self.r),:]
                if self.flatten:
                    patches[i] = patch.flatten()
                else:
                    patches[i]=patch.permute(2,0,1)
            self.patch_dict[ID]=patches
        data=[patches, positions, exps]
        if self.adj:
            data.append(adj)
        if self.ori:
            data+=[torch.Tensor(oris),torch.Tensor(sfs)]
        data.append(torch.Tensor(centers))
        return data
        
    def __len__(self):
        return len(self.exp_dict)

    def get_img(self,name):
        path = self.adata_dict[name].uns["spatial"][name]["metadata"]["source_image_path"]
        im = read_tiff(path)
        return im

In [1]:
import sys
sys.path.append("./")
# sys.path.append("../../scripts/")

import time

from window_adata import *
from read_stimage_genes import *

import torch
import numpy as np
import pytorch_lightning as pl
import torchvision.transforms as tf
from tqdm import tqdm
from predict import *
from HIST2ST import *
from dataset import ViT_HER2ST, ViT_SKIN
from scipy.stats import pearsonr,spearmanr
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from copy import deepcopy as dcp
import pickle
from collections import defaultdict as dfd
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score
import sys
from pathlib import Path
import os
import numpy as np
import pandas as pd
from pathlib import Path, PurePath
from typing import Union, Dict, Optional, Tuple, BinaryIO
import h5py
import json
from matplotlib.image import imread
import matplotlib.pyplot as plt

import anndata
from anndata import (
    AnnData,
    read_csv,
    read_text,
    read_excel,
    read_mtx,
    read_loom,
    read_hdf,
)
from anndata import read as read_h5ad
from anndata import read_h5ad
import scanpy as sc
from scanpy import read_visium, read_10x_mtx

import glob
import torch
import torchvision
import pandas as pd 
import scprep as scp
import anndata as ad
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None



def calculate_correlation(attr_1, attr_2):
    r = pearsonr(attr_1, 
                       attr_2)[0]
    return r


data_dir1 = "../../data/pfizer/"

samps1 = ["VLP79_D","VLP82_A","VLP79_A","VLP80_D","VLP83_A","VLP80_A","VLP81_A","VLP82_D","VLP83_D","VLP78_A"]

samples1 = {i:data_dir1 + i for i in samps1}

adata_dict1 = {name: read_visium(path, library_id = name, source_image_path = path + f"/image.tif") for name,path in samples1.items()}

adata_dict0 = {**adata_dict1}

for k,v in adata_dict0.items():
    v.obsm["spatial"] = v.obsm["spatial"].astype(np.int64)
    v.obs[['in_tissue','array_row','array_col']] = v.obs[['in_tissue','array_row','array_col']].astype(np.int64)


sizes = [3000 for i in range(len(adata_dict0))]

adata_dict = window_adata(adata_dict0, sizes)

# gene_list = read_gene_set("../../data/pfizer/") # all_adata.h5ad
gene_list = ['CD4', 'TRAC', 'CXCR4']
gene_list = set(gene_list)

gene_list = intersect_section_genes(gene_list, adata_dict)
n_genes = len(gene_list)

train_set = list(set(list(adata_dict.keys())) - set([i for i in list(adata_dict.keys()) if 'VLP78_A' in i]))


from data_vit import ViT_Anndata


device='cuda'
tag='5-7-2-8-4-16-32'
k,p,d1,d2,d3,h,c=map(lambda x:int(x),tag.split('-'))
dropout=0.2
random.seed(12000)
np.random.seed(12000)
torch.manual_seed(12000)
torch.cuda.manual_seed(12000)
torch.cuda.manual_seed_all(12000)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

df = pd.DataFrame()
i = int(sys.argv[1])
# i=1
fold = i
# test_sample = names[fold]




# gene_list = ["COX6C","TTLL12", "PABPC1", "GNAS", "HSP90AB1", 
#            "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]
# genes = len(gene_list)


trainset = ViT_Anndata(adata_dict = adata_dict, train_set = train_set, gene_list = gene_list,
            train=True,flatten=False,adj=True,ori=True,prune='NA',neighs=4, 
        )

print("LOADED TRAINSET")


train_loader = DataLoader(trainset, batch_size=1, num_workers=0, shuffle=True)

model=Hist2ST(
    depth1=d1, depth2=d2,depth3=d3,n_genes=n_genes,
    kernel_size=k, patch_size=p,
    heads=h, channel=c, dropout=0.2,
    zinb=0.25, nb=False,
    bake=5, lamb=0.5, n_pos=128,
)

logger=None
trainer = pl.Trainer(
    gpus=[0], max_epochs=1,
    logger=logger,
)

start_train = time.perf_counter()
trainer.fit(model, train_loader)

end_train = time.perf_counter()
import os
if not os.path.isdir("../../trained_models/"):
    os.mkdir("../../trained_models/")

torch.save(model.state_dict(),f"../trained_models/{fold}-Hist2ST.ckpt")


def evall(test_sample, gene_list):
    
    train_set = list(set(list(adata_dict.keys())) - set([test_sample]))
    testset = ViT_Anndata(adata_dict = adata_dict, train_set = train_set, gene_list = gene_list,
                train=False,flatten=False,adj=True,ori=True,prune='NA',neighs=4, 
            )
    test_loader = DataLoader(testset, batch_size=1, num_workers=0, shuffle=False)

    adata_pred, adata_truth = test(model, test_loader,'cuda')

    adata_pred.var_names = gene_list
    adata_truth.var_names = gene_list

    pred_adata = adata_pred.copy()
    test_dataset = adata_truth.copy()
    
    test_sample = ','.join(list(test_sample))
    
    with open(f"../../results/pf/hist2st_preds_{test_sample}_{i}.pkl", 'wb') as f:
        pickle.dump([pred_adata,test_dataset], f)

    for gene in pred_adata.var_names:
        cor_val = calculate_correlation(pred_adata.to_df().loc[:,gene], test_dataset.to_df().loc[:,gene])
        df = df.append(pd.Series([gene, cor_val, test_sample, "Hist2ST"], 
                             index=["Gene", "Pearson correlation", "Slide", "Method"]),
                  ignore_index=True)

    del model
    torch.cuda.empty_cache()

    df.to_csv("../../results/pf/hist2st_cor_{}_{i}.csv".format(test_sample, i))

    with open("../../results/pf/hist2st_times.txt", 'a') as f:
        f.write(f"{i} {test_sample} {end_train - start_train} - {time.strftime('%H:%M:%S', time.localtime())}")



evall([i for i in list(adata_dict.keys()) if 'VLP78_A' in i], gene_list)

# gene_list = ["COX6C","TTLL12", "HSP90AB1", 
#            "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]
# evall("CytAssist_FFPE", gene_list)


ModuleNotFoundError: No module named 'pytorch_lightning'

In [7]:
%%time
gene_list = ["COX6C","TTLL12", "PABPC1", "GNAS", "HSP90AB1", 
           "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]
train_set = list(set(list(adata_dict.keys())) - set(["FFPE", "CytAssist_FFPE"]))
ds = ViT_Anndata(adata_dict = adata_dict, train_set = train_set, gene_list = gene_list,
            train=True,flatten=False,adj=True,ori=True,prune='NA'
        )

Eval set:  ['CytAssist_FFPE', 'FFPE']
Loading imgs...
CPU times: user 1min 13s, sys: 32.2 s, total: 1min 46s
Wall time: 52.1 s


In [8]:
from torch.utils.data import DataLoader
train_loader = DataLoader(ds, batch_size=1, num_workers=0, shuffle=True)
iterator=iter(train_loader)

# for i in range(10):
#     print(i)
#     next(iterator)
#     print(ds.patch_dict.keys())
#     # print(ds.patch_dict)

In [9]:
test = next(iterator)

Index:  6


In [13]:
len(test)

7

In [50]:
for i,m in ds.exp_dict.items():
    print(m.shape)

(4992, 14)
(4992, 14)
(4992, 14)
(4992, 14)
(4992, 14)
(3798, 14)
(3987, 14)


In [46]:
for i in range(10):
    print(i)
    next(iterator)
    print([i.shape for i in test])

0
Index:  1
[torch.Size([1, 4992, 3, 112, 112]), torch.Size([1, 4992, 2]), torch.Size([1, 4992, 14]), torch.Size([1, 4992, 4992]), torch.Size([1, 4992, 14]), torch.Size([1, 4992]), torch.Size([1, 4992, 2])]
1
Index:  2
[torch.Size([1, 4992, 3, 112, 112]), torch.Size([1, 4992, 2]), torch.Size([1, 4992, 14]), torch.Size([1, 4992, 4992]), torch.Size([1, 4992, 14]), torch.Size([1, 4992]), torch.Size([1, 4992, 2])]
2
Index:  5
[torch.Size([1, 4992, 3, 112, 112]), torch.Size([1, 4992, 2]), torch.Size([1, 4992, 14]), torch.Size([1, 4992, 4992]), torch.Size([1, 4992, 14]), torch.Size([1, 4992]), torch.Size([1, 4992, 2])]
3
Index:  3
[torch.Size([1, 4992, 3, 112, 112]), torch.Size([1, 4992, 2]), torch.Size([1, 4992, 14]), torch.Size([1, 4992, 4992]), torch.Size([1, 4992, 14]), torch.Size([1, 4992]), torch.Size([1, 4992, 2])]
4
Index:  0
[torch.Size([1, 4992, 3, 112, 112]), torch.Size([1, 4992, 2]), torch.Size([1, 4992, 14]), torch.Size([1, 4992, 4992]), torch.Size([1, 4992, 14]), torch.Size([1,

StopIteration: 

In [25]:
ds.img_dict['1142243F'].dtype

torch.float32

In [24]:
ds.patch_dict['CID44971'].dtype

torch.float32

In [26]:
ds.center_dict['CID44971'].dtype

dtype('int64')

In [32]:
ds.loc_dict['CID44971']

array([[ 16,   0],
       [102,  50],
       [ 43,   3],
       ...,
       [ 27,  45],
       [ 41,  73],
       [ 51,   7]])

In [38]:
ds.adj_dict['CID44971'].count_nonzero() / ds.adj_dict['CID44971'].shape[0]

tensor(4.)

In [45]:
test[1][:,:,0]

tensor([[ 16, 102,  43,  ...,  27,  41,  51]])